In [22]:
import os
import struct
import sys

try:
    from PIL import Image
except:
    print ('You do not have PIL (the Python Imaging Library) installed.')
    sys.exit(1)

class SeamError(Exception):
    pass

class ImageMatrix(dict):
    def __init__(self, image):
        """Takes either a PIL image, or a filename of an image. Stores
        pixels in its dictionary, and also stores width and height."""
        if not isinstance(image, Image.Image):
            image = Image.open(image)
        self.width, self.height = image.size
        pixels = iter(image.getdata())
        for j in range(self.height):
            for i in range(self.width):
                self[i,j] = next(pixels)

    def color_seam(self, seam, color=(255,0,0)):
        """Takes a seam (a list of coordinates) and colors it all one
        color."""
        for i,j in seam:
            self[i,j] = color

    def remove_seam(self, seam):
        """Takes a seam (a list of coordinates with exactly one pair of
        coordinates per row). Removes pixel at each of those coordinates,
        and slides left all the pixels to its right. Decreases the width
        by 1."""
        seen = [False for j in range(self.height)]
        for i,j in seam:
            if not (0 <= j < self.height):
                raise SeamError('seam has nonexistent row %d' % i)
            if seen[j]:
                raise SeamError('seam has repeated row %d' % i)
            seen[j] = True
            for ii in range(i, self.width-1):
                self[ii,j] = self[ii+1,j]
            del self[self.width-1,j]
        missed = [j for j in range(self.height) if not seen[j]]
        if missed:
            raise SeamError('seam missed rows %s' % ','.join(map(str,missed)))
        self.width -= 1

    def image(self):
        """Returns a PIL Image that is represented by self."""
        image = Image.new('RGB', (self.width, self.height))
        image.putdata(
            [self[i,j] for j in range(self.height) for i in range(self.width)])
        return image

    def save(self,*args,**keyw):
        self.image().save(*args,**keyw)

    def ppm(self):
        """Returns self in (binary) ppm form."""
        return b'P6 %d %d 255\n' % (self.width, self.height) + \
            b''.join([struct.pack("BBB", *self[i,j])
                      for j in range(self.height) for i in range(self.width)])

    def save_ppm(self, filename):
        """Saves self as a .ppm"""
        f = open(filename, 'wb')
        f.write(self.ppm())
        f.close() 

    def show(self,title='image',temp='_temp_.ppm'):
        """Displays self in a pop-up window using Tkinter,
        and waits till the user either clicks on or closes the window.
        Saves the image as a temporary ppm file (specified by temp)."""
        import Tkinter
        if Tkinter._default_root:
            root=Tkinter.Toplevel()
        else:
            root=Tkinter.Tk()
        self.save_ppm(temp)
        image = Tkinter.PhotoImage(master=root, file=temp)
        root.title('%dx%d image' % (self.width, self.height))
        label = Tkinter.Label(root, image=image)
        label.pack()
        label.bind('<Button>', lambda e: root.destroy())
        root.mainloop()
        os.remove(temp)

    def energy(self, i, j):
        """Given coordinates (i,j), returns an energy, or cost associated
        with removing that pixel."""
        if i==0 or j==0 or i==self.width-1 or j==self.height-1:
            # For simplicity, return an arbitrarily large value on the edge.
            return 10000
        else: # I think this is equivalent to the Sobel gradient magnitude.
            return self.distance(self[i-1,j], self[i+1,j]) +\
                   self.distance(self[i,j-1], self[i,j+1]) +\
                   self.distance(self[i-1,j-1], self[i+1,j+1]) +\
                   self.distance(self[i+1,j-1], self[i-1,j+1])

    def distance(self, pixelA, pixelB):
        """A distance metric between two pixels, based on their colors."""
        ans = 0
        for i in range(len(pixelA)):
            valueA = pixelA[i]
            valueB = pixelB[i]
            ans += abs(valueA-valueB)
        return ans

In [23]:
import sys

class ResizeableImage(ImageMatrix):
    
    def best_seam(self):
       
        energy = {}
        dp = {} #solution to subproblems
        backpointer = {} #stores the pixels in the lowest seam
        seam = [] #result
        
        for i in range(self.width):  #Calculate energy for each pixel in image
            for j in range(self.height):
                energy[i,j] = self.energy(i,j)

        for i in range(self.width): 
            dp[i,0] = energy[i,0]
        
        for j in range(1, self.height):
            for i in range(self.width): #proceed down the image
                
                dp[i,j] = dp[i,j-1] + energy[i,j]
                
                backpointer[i,j] = 0
                
                #look at path of pixels to the right
                if i != self.width-1:
                    if dp[i,j] >= dp[i+1,j-1]+energy[i,j]: 
                        backpointer[i,j] = 1
                        dp[i,j] = dp[i+1,j-1]+energy[i,j]
                
                #look at path of pixels to the left
                if i != 0:
                    if dp[i,j] >= dp[i-1,j-1]+energy[i,j]:
                        backpointer[i,j] = -1
                        dp[i,j] = dp[i-1,j-1]+energy[i,j]
                        
        i= None
        least_energy = float("inf")
        for j in range(self.width):
            if dp[j,self.height-1] <= least_energy:
                i= j
                least_energy = dp[j,self.height-1]

        for j in range(self.height-1, 0, -1):
            seam.append((i, j))
            i = i + backpointer[i,j]
        seam.append((i, 0))
        return seam

    def remove_best_seam(self):
        self.remove_seam(self.best_seam())

In [24]:
import os
from tkinter import *
import tkinter.filedialog
from PIL import Image

seam = None
image = None
temp= '_gtemp_.ppm'

def open_file():
    global image, status
    filename = tkinter.filedialog.askopenfilename()
    if filename is None: return
    status['text'] = 'Loading %s...' % os.path.basename(filename)
    status.update()
    try:
        image = ResizeableImage(filename)
    except:
        status['text'] = 'Error loading %s!' % os.path.basename(filename)
        raise
    update_display()
    seam = None
    status['text'] = 'Loaded %s.  Now compute or remove seam.' % \
                     os.path.basename(filename)

def save_file():
    global image, status
    if image is None: return
    filename = tkinter.filedialog.asksaveasfilename()
    if filename is None: return
    status['text'] = 'Saving %s...' % os.path.basename(filename)
    status.update()
    try:
        image.save(filename)
    except:
        status['text'] = 'Error saving %s!' % os.path.basename(filename)
        raise
    status['text'] = 'Saved %s.' % os.path.basename(filename)

def update_display():
    global image, photo, display, root, buttons
    image.save_ppm(temp)
    photo = tkinter.PhotoImage(master=root, file=temp)
    display['image'] = photo
    root.wm_geometry('%dx%d' % (
      buttons.winfo_width() + image.width,
      status.winfo_height() + max(image.height, buttons.winfo_height()) ))
    os.remove(temp)

def compute_seam(count=0):
    global seam
    if seam is None:
        if count:
            status['text'] = 'Computing seam %d...' % (count+1)
        else:
            status['text'] = 'Computing seam...'
        status.update()
        seam = image.best_seam()
        if count:
            status['text'] = 'Computed seam %d.' % (count+1)
        else:
            status['text'] = 'Computed seam.'

def show_seam():
    global image, seam
    if image is None: return
    compute_seam()
    image.color_seam(seam)
    update_display()
    status['text'] = 'Computed seam, as shown in red.'

def remove_seam():
    global image, seam
    if image is None: return
    count = 0
    while True:
        compute_seam(count)
        image.remove_seam(seam)
        update_display()
        seam = None
        count += 1

        try:
            repeat = int(multiple_spin.get())
        except ValueError:
            break
        if repeat <= 1: break
        repeat -= 1
        multiple_spin.delete(0,'end')
        multiple_spin.insert(0,repeat)
        multiple_spin.update()
    multiple_spin.delete(0,'end')
    multiple_spin.insert(0,1)
    if count > 1:
        status['text'] = 'Removed %d seams.' % count
    else:
        status['text'] = 'Removed seam.'

root = tkinter.Tk()
root.title('6.006 Seam Carving')
status = tkinter.Label(text='Please open an image.')
status.pack(side='top')
buttons = tkinter.Frame()
open_button = tkinter.Button(buttons, text='Open...', command=open_file)
open_button.pack(side='top', fill='x')
save_button = tkinter.Button(buttons, text='Save...', command=save_file)
save_button.pack(side='top', fill='x')
show_button = tkinter.Button(buttons, text='Show Seam', command=show_seam)
show_button.pack(side='top', fill='x')
remove_button = tkinter.Button(buttons, text='Remove Seam', command=remove_seam)
remove_button.pack(side='top', fill='x')
multiple_frame = tkinter.Frame(buttons)
multiple_label = tkinter.Label(multiple_frame, text='Repeat:')
multiple_label.pack(side='left')
multiple_spin = tkinter.Spinbox(multiple_frame,
    width=3, from_=1, to_=100, increment=1)
multiple_spin.pack(side='right')
multiple_frame.pack(side='top', fill='x')
buttons.pack(side='left')
display = tkinter.Label(root)
display.pack(side='top')
root.mainloop()
